# Feature Exploration in the European soccer dataset

In [1]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from train_test.general import train_test_bookies_comparison, display_train_test_bookies_comparison
from feature_eng.odds import bookie_prediction, prob_by_bookies
from feature_eng.team_stats import goal_stats, elo_scores, glicko2_scores, trueskill_scores
import sqlite3
import pandas as pd
import os
import numpy as np

## Retrieval of matchs dataset

In [2]:
# Retrieval pf the matchs dataset
DATA_PATH = f'./../../data/soccer/European_Soccer_Database/database.sqlite'
conn = sqlite3.connect(DATA_PATH)
tables = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table'", conn)
matchs = pd.read_sql("SELECT * FROM Match", conn)
tables

,name
0,sqlite_sequence
1,Player_Attributes
2,Player
3,Match
4,League
5,Country
6,Team
7,Team_Attributes


```python
{ bookie : { "model_mean_all" : { metric : value }, 
                "model_all" : { metric : [value] }, 
                "bookie_mean" : { metric : value }, 
                "bookie" : { metric : [value] } } 
                }

metric = ["accuracy", "weighted_accuracy", "accuracy_home", "accuracy_draw", "accuracy_away",
            "recall_all", "weighted_recall", "balanced_accuracy", "recall_home", "recall_draw", "recall_away",
            "precision_all", "weighted_precision", "precision_home", "precision_draw", "precision_away",
            "f_mesure_all", "f_mesure_weighted", "f_mesure_home", "f_mesure_draw", "f_mesure_away",
            "log_loss", "loss_home", "loss_draw", "loss_away",
            "mse", "mse_home", "mse_draw", "mse_away",
            "classwise_ECE", "ECE_home", "ECE_draw", "ECE_away", "home_ECE_y", "draw_ECE_y", "away_ECE_y", "home_ECE_p", "draw_ECE_p", "away_ECE_p", "home_ECE_size", "draw_ECE_size", "away_ECE_size"]
```


## Feature engineering

In [3]:
# Features enginnering (minimun col to add to be able to test the prediction of the bookmakers)
matchs['FTR'] = matchs.apply(lambda x: 1 if x['home_team_goal'] > x['away_team_goal'] else 0 if x['home_team_goal'] == x['away_team_goal'] else -1, axis=1)
bookies = ['B365', 'BW', 'IW', 'LB', 'PS', 'WH', 'SJ', 'VC', 'GB', 'BS']
for bookie in bookies:
    matchs[f'{bookie}_prediction'] = matchs.apply(lambda x: bookie_prediction(x, bookie), axis=1)
for bookie in bookies:
    matchs[f'{bookie}H_prob'], matchs[f'{bookie}D_prob'], matchs[f'{bookie}A_prob'] = zip(*matchs.apply(lambda x: prob_by_bookies(x, bookie), axis=1))

# Replacing missing odd by the mean of the other bookies
for bookie in bookies:
    matchs[f'{bookie}H'] = matchs[f'{bookie}H'].fillna(matchs[f'{bookie}H'].mean())
    matchs[f'{bookie}D'] = matchs[f'{bookie}D'].fillna(matchs[f'{bookie}D'].mean())
    matchs[f'{bookie}A'] = matchs[f'{bookie}A'].fillna(matchs[f'{bookie}A'].mean())
    matchs[f'{bookie}H_prob'] = matchs[f'{bookie}H_prob'].fillna(matchs[f'{bookie}H_prob'].mean())
    matchs[f'{bookie}D_prob'] = matchs[f'{bookie}D_prob'].fillna(matchs[f'{bookie}D_prob'].mean())
    matchs[f'{bookie}A_prob'] = matchs[f'{bookie}A_prob'].fillna(matchs[f'{bookie}A_prob'].mean())

In [4]:
elo_scores(matchs)
goal_stats(matchs)
glicko2_scores(matchs)
trueskill_scores(matchs)

## Comparison elo, glick2, goal stats, true skill one by one Logistic regression

In [5]:
# Feature selection
odds_cols = ("odds", [el for el_tuple in [(f"{bookie}H", f"{bookie}D", f"{bookie}A") for bookie in bookies] for el in el_tuple])
prob_cols = ("prob", [el for el_tuple in [(f"{bookie}H_prob", f"{bookie}D_prob", f"{bookie}A_prob") for bookie in bookies] for el in el_tuple])
elo_scores_cols = ("elo_scores", ['elo_home_before', 'elo_away_before'])
goal_stats_cols = ("goal_stats", ["home_team_goals_season_to_date_before_match", "avg_away_team_goals_season_to_date_before_match"])
matchs_stats_cols = ("matchs_stats", ["home_team_number_of_match_played", "away_team_number_of_match_played"])
glicko2_scores_cols = ("glicko2_scores", ["glicko2_home_before", "glicko2_away_before"])
glicko2_all_scores_cols = ("glicko2_all_scores", ["glicko2_home_before", "glicko2_away_before", "glicko2_rd_home_before", "glicko2_rd_away_before", "glicko2_vol_home_before", "glicko2_vol_away_before"])
trueskill_scores_cols = ("trueskill_scores", ["trueskill_home_before", "trueskill_away_before"])


In [6]:
# Test of the function
X_cols = [odds_cols, prob_cols, elo_scores_cols, goal_stats_cols, matchs_stats_cols, 
            glicko2_scores_cols, glicko2_all_scores_cols, trueskill_scores_cols]
Y_col = "FTR"
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('model', LogisticRegression())
])
metrics_bookies = train_test_bookies_comparison(matchs, pipeline, X_cols, Y_col)
df_display = display_train_test_bookies_comparison(metrics_bookies, X_cols, metrics_of_interrest=["accuracy", "log_loss", "mse"])

c:\Users\jdelavande\Projects\optim-sportbet\venv\lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\jdelavande\Projects\optim-sportbet\venv\lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules

In [7]:
df_display_acc = display_train_test_bookies_comparison(metrics_bookies, X_cols, metrics_of_interrest=["accuracy"])
df_display_acc

,B365,BW,IW,LB,PS,WH,SJ,VC,GB,BS,all_df
accuracy,0.543619,0.542757,0.547482,0.544965,0.515888,0.543831,0.529122,0.544820,0.527374,0.526394,0.536625
accuracy_model_odds,0.540835,0.539486,0.546763,0.543794,0.511215,0.543596,0.526348,0.539464,0.525512,0.520818,0.520818
accuracy_model_prob,0.544316,0.543925,0.547962,0.547541,0.509813,0.541951,0.527581,0.545518,0.515829,0.520074,0.520074
accuracy_model_elo_scores,0.503016,0.503037,0.505276,0.506557,0.474766,0.501528,0.503236,0.502678,0.508380,0.507807,0.507807
accuracy_model_goal_stats,0.484455,0.484346,0.487770,0.486183,0.474299,0.485076,0.483821,0.484750,0.469646,0.468773,0.468773
accuracy_model_matchs_stats,0.469374,0.469393,0.472182,0.472600,0.435047,0.468860,0.470878,0.470547,0.454004,0.453532,0.453532
accuracy_model_glicko2_scores,0.522506,0.521729,0.527098,0.525527,0.498131,0.521974,0.511248,0.523632,0.511359,0.510037,0.510037
accuracy_model_glicko2_all_scores,0.519490,0.519159,0.522542,0.522014,0.500467,0.519624,0.510940,0.519907,0.502048,0.501115,0.501115
accuracy_model_trueskill_scores,0.516473,0.516121,0.519904,0.519672,0.496262,0.518449,0.517103,0.516182,0.505400,0.504833,0.504833


In [8]:
df_display_log_loss = display_train_test_bookies_comparison(metrics_bookies, X_cols, metrics_of_interrest=["log_loss"])
df_display_log_loss

,B365,BW,IW,LB,PS,WH,SJ,VC,GB,BS,all_df
log_loss,0.962041,0.964449,0.962369,0.962427,0.980963,0.964644,0.974443,0.961478,0.975106,0.975457,0.968338
log_loss_model_odds,0.969428,0.969567,0.966599,0.967423,0.986624,0.970377,0.978317,0.969007,0.982726,0.985208,0.985208
log_loss_model_prob,0.965117,0.968173,0.964001,0.963556,0.985637,0.967119,0.979276,0.965110,0.987887,0.988654,0.988654
log_loss_model_elo_scores,1.016467,1.016551,1.014938,1.013902,1.034781,1.018117,1.014418,1.015932,1.016319,1.016810,1.016810
log_loss_model_goal_stats,1.035529,1.034608,1.032835,1.033132,1.042633,1.035008,1.038466,1.034663,1.042423,1.043372,1.043372
log_loss_model_matchs_stats,1.058623,1.058502,1.056976,1.056913,1.076788,1.058966,1.057746,1.057872,1.066646,1.066860,1.066860
log_loss_model_glicko2_scores,0.992841,0.992501,0.990140,0.989757,0.997449,0.992853,0.997493,0.991280,1.008002,1.008548,1.008548
log_loss_model_glicko2_all_scores,0.997406,0.996814,0.995486,0.994636,0.997309,0.997262,1.002058,0.995752,1.019753,1.020924,1.020924
log_loss_model_trueskill_scores,1.013536,1.014048,1.011788,1.012347,1.016786,1.012763,1.009803,1.012967,1.024293,1.024553,1.024553


In [9]:
df_display_classwise_ece = display_train_test_bookies_comparison(metrics_bookies, X_cols, metrics_of_interrest=["classwise_ECE"])
df_display_classwise_ece

,B365,BW,IW,LB,PS,WH,SJ,VC,GB,BS,all_df
classwise_ECE,0.028777,0.028306,0.029315,0.029177,0.044381,0.028145,0.031270,0.027190,0.035880,0.036486,0.031893
classwise_ECE_model_odds,0.033259,0.034185,0.034614,0.035039,0.044100,0.031258,0.030027,0.034114,0.037104,0.036969,0.036969
classwise_ECE_model_prob,0.030868,0.033841,0.033332,0.035467,0.041129,0.031982,0.033329,0.033434,0.041349,0.039619,0.039619
classwise_ECE_model_elo_scores,0.028732,0.031131,0.030341,0.029643,0.041406,0.029687,0.030799,0.028792,0.038634,0.037112,0.037112
classwise_ECE_model_goal_stats,0.038594,0.037563,0.038299,0.038134,0.047467,0.037549,0.038715,0.038008,0.044065,0.045529,0.045529
classwise_ECE_model_matchs_stats,0.014725,0.014804,0.014631,0.014606,0.021364,0.013775,0.010515,0.014260,0.013689,0.012530,0.012530
classwise_ECE_model_glicko2_scores,0.031673,0.032164,0.033368,0.032798,0.044902,0.031146,0.028109,0.032311,0.040892,0.040426,0.040426
classwise_ECE_model_glicko2_all_scores,0.033749,0.034650,0.034906,0.033526,0.042674,0.032486,0.030173,0.033590,0.047279,0.047106,0.047106
classwise_ECE_model_trueskill_scores,0.040521,0.040658,0.042563,0.042094,0.047695,0.041055,0.035171,0.040710,0.043958,0.044266,0.044266


In [10]:
df_display_bal_acc = display_train_test_bookies_comparison(metrics_bookies, X_cols, metrics_of_interrest=["balanced_accuracy"])
df_display_bal_acc

,B365,BW,IW,LB,PS,WH,SJ,VC,GB,BS,all_df
balanced_accuracy,0.450131,0.448807,0.450231,0.449198,0.446973,0.450899,0.437203,0.449797,0.443816,0.443603,0.447066
balanced_accuracy_model_odds,0.445535,0.442778,0.447828,0.445609,0.442640,0.448217,0.430485,0.442494,0.442097,0.437443,0.437443
balanced_accuracy_model_prob,0.455365,0.455033,0.455934,0.457243,0.447743,0.453278,0.439481,0.454676,0.435946,0.440986,0.440986
balanced_accuracy_model_elo_scores,0.395944,0.395878,0.394764,0.396830,0.399519,0.394788,0.395929,0.394584,0.411191,0.411593,0.411593
balanced_accuracy_model_goal_stats,0.381692,0.380987,0.382957,0.382025,0.397426,0.381767,0.379383,0.381429,0.355824,0.355180,0.355180
balanced_accuracy_model_matchs_stats,0.333333,0.333333,0.333333,0.333333,0.333333,0.333333,0.333333,0.333333,0.333333,0.333333,0.333333
balanced_accuracy_model_glicko2_scores,0.420670,0.418963,0.422626,0.421754,0.430099,0.420234,0.409027,0.420463,0.419640,0.418879,0.418879
balanced_accuracy_model_glicko2_all_scores,0.420661,0.419728,0.420984,0.421053,0.432470,0.420579,0.411937,0.419874,0.415516,0.415555,0.415555
balanced_accuracy_model_trueskill_scores,0.429546,0.428117,0.430707,0.431146,0.426920,0.431684,0.420973,0.428123,0.415416,0.415589,0.415589


## XGBOOST

In [13]:
from xgboost import XGBClassifier
#0 if FTR = 1, 1 if FTR = 0, 2 if FTR = -1
matchs['FTR_positif'] = matchs['FTR'].apply(lambda x: 0 if x == 1 else 1 if x == 0 else 2)
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('model', XGBClassifier())
])
metrics_bookies_xgb = train_test_bookies_comparison(matchs, pipeline, X_cols, Y_col='FTR_positif')
df_display_acc_xgb = display_train_test_bookies_comparison(metrics_bookies_xgb, X_cols, metrics_of_interrest=["accuracy"])
df_display_acc_xgb

,B365,BW,IW,LB,PS,WH,SJ,VC,GB,BS,all_df
accuracy,0.543619,0.542757,0.547482,0.544965,0.515888,0.543831,0.529122,0.544820,0.527374,0.526394,0.536625
accuracy_model_odds,0.201624,0.197664,0.201199,0.200234,0.194393,0.204935,0.204931,0.193714,0.217505,0.216357,0.216357
accuracy_model_prob,0.199072,0.199766,0.206954,0.193443,0.202336,0.203995,0.201849,0.197672,0.213035,0.220818,0.220818
accuracy_model_elo_scores,0.203248,0.213551,0.208393,0.202810,0.210280,0.203525,0.207396,0.205355,0.217877,0.212639,0.212639
accuracy_model_goal_stats,0.207193,0.206776,0.203837,0.206792,0.226168,0.209166,0.212018,0.206985,0.231285,0.238662,0.238662
accuracy_model_matchs_stats,0.249652,0.248832,0.248681,0.248712,0.254673,0.250764,0.249923,0.247963,0.252514,0.252045,0.252045
accuracy_model_glicko2_scores,0.202784,0.197664,0.193765,0.195316,0.213084,0.196475,0.207088,0.197206,0.219367,0.216357,0.216357
accuracy_model_glicko2_all_scores,0.202320,0.209346,0.206475,0.213349,0.200000,0.210811,0.210786,0.203725,0.212291,0.212268,0.212268
accuracy_model_trueskill_scores,0.199768,0.201636,0.196163,0.202810,0.195794,0.203760,0.200616,0.200233,0.207076,0.212268,0.212268


In [14]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
#0 if FTR = 1, 1 if FTR = 0, 2 if FTR = -1
matchs['FTR_positif'] = matchs['FTR'].apply(lambda x: 0 if x == 1 else 1 if x == 0 else 2)
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('model', RandomForestClassifier())
])
metrics_bookies_xgb = train_test_bookies_comparison(matchs, pipeline, X_cols, Y_col='FTR_positif')
df_display_acc_xgb = display_train_test_bookies_comparison(metrics_bookies_xgb, X_cols, metrics_of_interrest=["accuracy"])
df_display_acc_xgb

,B365,BW,IW,LB,PS,WH,SJ,VC,GB,BS,all_df
accuracy,0.543619,0.542757,0.547482,0.544965,0.515888,0.543831,0.529122,0.544820,0.527374,0.526394,0.536625
accuracy_model_odds,0.201160,0.193692,0.206235,0.192037,0.200467,0.202350,0.193220,0.189523,0.213035,0.215985,0.210037
accuracy_model_prob,0.191879,0.195093,0.198082,0.187588,0.203271,0.199530,0.199384,0.194179,0.211918,0.212639,0.205948
accuracy_model_elo_scores,0.213457,0.212850,0.206235,0.220141,0.224299,0.208931,0.208629,0.213737,0.215642,0.216729,0.210037
accuracy_model_goal_stats,0.216241,0.215187,0.217026,0.219672,0.237850,0.212926,0.218490,0.217462,0.228678,0.221561,0.221933
accuracy_model_matchs_stats,0.246404,0.249766,0.248921,0.244262,0.259813,0.251469,0.251156,0.247963,0.252514,0.252788,0.252788
accuracy_model_glicko2_scores,0.216241,0.207710,0.202398,0.205855,0.209346,0.213631,0.215408,0.208847,0.221601,0.227881,0.217844
accuracy_model_glicko2_all_scores,0.206032,0.204206,0.211031,0.203981,0.196729,0.201410,0.200924,0.200931,0.210056,0.208922,0.211896
accuracy_model_trueskill_scores,0.213689,0.211916,0.216547,0.213583,0.222897,0.215276,0.213867,0.210477,0.217505,0.215242,0.210781


In [15]:
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('model', RandomForestClassifier())
])
metrics_bookies_xgb = train_test_bookies_comparison(matchs, pipeline, X_cols, Y_col='FTR')
df_display_acc_xgb = display_train_test_bookies_comparison(metrics_bookies_xgb, X_cols, metrics_of_interrest=["accuracy"])
df_display_acc_xgb

,B365,BW,IW,LB,PS,WH,SJ,VC,GB,BS,all_df
accuracy,0.543619,0.542757,0.547482,0.544965,0.515888,0.543831,0.529122,0.544820,0.527374,0.526394,0.536625
accuracy_model_odds,0.509745,0.506075,0.523981,0.508197,0.487850,0.508343,0.511556,0.518510,0.479702,0.478810,0.481784
accuracy_model_prob,0.508353,0.504907,0.515348,0.518735,0.500467,0.510928,0.513713,0.506170,0.493110,0.491450,0.491450
accuracy_model_elo_scores,0.426914,0.435047,0.429496,0.435363,0.403738,0.433608,0.434206,0.431898,0.441341,0.444610,0.435316
accuracy_model_goal_stats,0.411137,0.409579,0.407434,0.408665,0.398598,0.419976,0.409245,0.412806,0.418994,0.417100,0.411152
accuracy_model_matchs_stats,0.467749,0.467056,0.471463,0.471897,0.422430,0.467215,0.469029,0.469150,0.446182,0.444610,0.444981
accuracy_model_glicko2_scores,0.432019,0.432477,0.441487,0.437705,0.442991,0.428907,0.439137,0.433760,0.428678,0.436059,0.428996
accuracy_model_glicko2_all_scores,0.479118,0.487383,0.487290,0.490867,0.467290,0.486016,0.479507,0.485914,0.477467,0.481784,0.475836
accuracy_model_trueskill_scores,0.451740,0.446495,0.450360,0.447541,0.454206,0.439248,0.457935,0.452386,0.441713,0.441264,0.443866


In [16]:
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('model', GradientBoostingClassifier())
])
metrics_bookies_xgb = train_test_bookies_comparison(matchs, pipeline, X_cols, Y_col='FTR')
df_display_acc_xgb = display_train_test_bookies_comparison(metrics_bookies_xgb, X_cols, metrics_of_interrest=["accuracy"])
df_display_acc_xgb

,B365,BW,IW,LB,PS,WH,SJ,VC,GB,BS,all_df
accuracy,0.543619,0.542757,0.547482,0.544965,0.515888,0.543831,0.529122,0.544820,0.527374,0.526394,0.536625
accuracy_model_odds,0.531090,0.529673,0.542446,0.536300,0.510280,0.534665,0.522034,0.535041,0.503538,0.507063,0.507063
accuracy_model_prob,0.529930,0.524533,0.533813,0.535129,0.505607,0.530670,0.513097,0.531548,0.508380,0.510037,0.512268
accuracy_model_elo_scores,0.499536,0.499065,0.500240,0.499766,0.459346,0.496592,0.489985,0.500815,0.495717,0.492937,0.492937
accuracy_model_goal_stats,0.492343,0.494860,0.497602,0.499297,0.473832,0.496122,0.488136,0.492200,0.482682,0.482900,0.482900
accuracy_model_matchs_stats,0.468677,0.468458,0.471703,0.472131,0.422430,0.468390,0.470878,0.469849,0.449534,0.444981,0.444981
accuracy_model_glicko2_scores,0.521810,0.520327,0.526859,0.524122,0.488318,0.525499,0.514946,0.525262,0.503911,0.500743,0.500743
accuracy_model_glicko2_all_scores,0.522970,0.528505,0.530216,0.527400,0.500000,0.528555,0.510324,0.525960,0.503166,0.494424,0.494796
accuracy_model_trueskill_scores,0.521578,0.521963,0.528297,0.521077,0.494860,0.523854,0.515562,0.526193,0.502048,0.497770,0.497770


In [17]:
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('model', AdaBoostClassifier())
])
metrics_bookies_xgb = train_test_bookies_comparison(matchs, pipeline, X_cols, Y_col='FTR')
df_display_acc_xgb = display_train_test_bookies_comparison(metrics_bookies_xgb, X_cols, metrics_of_interrest=["accuracy"])
df_display_acc_xgb

c:\Users\jdelavande\Projects\optim-sportbet\venv\lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
c:\Users\jdelavande\Projects\optim-sportbet\venv\lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
c:\Users\jdelavande\Projects\optim-sportbet\venv\lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
c:\Users\jdelavande\Projects\optim-sportbet\venv\lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be r

,B365,BW,IW,LB,PS,WH,SJ,VC,GB,BS,all_df
accuracy,0.543619,0.542757,0.547482,0.544965,0.515888,0.543831,0.529122,0.544820,0.527374,0.526394,0.536625
accuracy_model_odds,0.534339,0.529206,0.538849,0.538642,0.515421,0.533020,0.520493,0.534575,0.502421,0.511524,0.511524
accuracy_model_prob,0.522042,0.526869,0.521583,0.523653,0.512150,0.527380,0.527889,0.527357,0.515456,0.511524,0.511524
accuracy_model_elo_scores,0.498376,0.498832,0.501199,0.502576,0.474766,0.493537,0.495840,0.498719,0.491620,0.488848,0.488848
accuracy_model_goal_stats,0.491879,0.492523,0.493765,0.497190,0.471028,0.493772,0.493991,0.498021,0.494600,0.491822,0.491822
accuracy_model_matchs_stats,0.469374,0.469393,0.472182,0.472600,0.422430,0.468860,0.470878,0.470547,0.448417,0.449442,0.449442
accuracy_model_glicko2_scores,0.520882,0.522664,0.526139,0.525293,0.491589,0.522209,0.510324,0.523865,0.510987,0.508178,0.508178
accuracy_model_glicko2_all_scores,0.519954,0.516355,0.518225,0.521546,0.490187,0.520094,0.498305,0.518277,0.496462,0.495539,0.495539
accuracy_model_trueskill_scores,0.517401,0.517056,0.522782,0.521546,0.493458,0.521739,0.513097,0.521769,0.497207,0.492565,0.492565


## Merging player stats


In [18]:
player_attributes = pd.read_sql_query("SELECT * from Player_Attributes", conn)

In [19]:
player_attributes.head()

,id,player_fifa_api_id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,1,218353,505942,2016-02-18 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
1,2,218353,505942,2015-11-19 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
2,3,218353,505942,2015-09-21 00:00:00,62.0,66.0,right,medium,medium,49.0,...,54.0,48.0,65.0,66.0,69.0,6.0,11.0,10.0,8.0,8.0
3,4,218353,505942,2015-03-20 00:00:00,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
4,5,218353,505942,2007-02-22 00:00:00,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0


In [21]:
home_player_cols = [f"home_player_{i}" for i in range(1, 12)]
away_player_cols = [f"away_player_{i}" for i in range(1, 12)]
player_attributes = player_attributes.sort_values('date', ascending=False)

df_match = matchs.copy()
df_joueur = player_attributes.copy()

for joueur in home_player_cols:
    df_temp = pd.merge(df_match, df_joueur,  how='outer', left_on=[joueur], right_on = ['player_api_id'], suffixes=('', '_'+joueur))
    df_temp.sort_values('date', inplace=True)
    df_temp.fillna(method='ffill', inplace=True)
    df_match = df_temp[df_temp['date'].notna()]

for joueur in away_player_cols:
    df_temp = pd.merge(df_match, df_joueur,  how='outer', left_on=[joueur], right_on = ['player_api_id'], suffixes=('', '_'+joueur))
    df_temp.sort_values('date', inplace=True)
    df_temp.fillna(method='ffill', inplace=True)
    df_match = df_temp[df_temp['date'].notna()]

C:\Users\jdelavande\AppData\Local\Temp\ipykernel_23232\1387973081.py:11: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_temp.fillna(method='ffill', inplace=True)


MemoryError: Unable to allocate 21.2 GiB for an array with shape (201, 14164130) and data type float64

In [27]:
df_match = matchs.iloc[:100].copy()
df_joueur = player_attributes.iloc[:1000].copy()

In [28]:
for joueur in home_player_cols:
    df_temp = pd.merge(df_match, df_joueur,  how='outer', left_on=[joueur], right_on = ['player_api_id'], suffixes=('', '_'+joueur))
    df_temp.sort_values('date', inplace=True)
    df_temp.fillna(method='ffill', inplace=True)
    df_match = df_temp[df_temp['date'].notna()]

for joueur in away_player_cols:
    df_temp = pd.merge(df_match, df_joueur,  how='outer', left_on=[joueur], right_on = ['player_api_id'], suffixes=('', '_'+joueur))
    df_temp.sort_values('date', inplace=True)
    df_temp.fillna(method='ffill', inplace=True)
    df_match = df_temp[df_temp['date'].notna()]

C:\Users\jdelavande\AppData\Local\Temp\ipykernel_23232\2059502095.py:4: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_temp.fillna(method='ffill', inplace=True)
C:\Users\jdelavande\AppData\Local\Temp\ipykernel_23232\2059502095.py:4: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_temp.fillna(method='ffill', inplace=True)
C:\Users\jdelavande\AppData\Local\Temp\ipykernel_23232\2059502095.py:4: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_temp.fillna(method='ffill', inplace=True)
C:\Users\jdelavande\AppData\Local\Temp\ipykernel_23232\2059502095.py:4: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_temp.fillna(method='ff

In [29]:
df_match

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,vision_away_player_11,penalties_away_player_11,marking_away_player_11,standing_tackle_away_player_11,sliding_tackle_away_player_11,gk_diving_away_player_11,gk_handling_away_player_11,gk_kicking_away_player_11,gk_positioning_away_player_11,gk_reflexes_away_player_11
1000,2.0,1.0,1.0,2008/2009,1.0,2008-08-16 00:00:00,492474.0,10000.0,9994.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1001,3.0,1.0,1.0,2008/2009,1.0,2008-08-16 00:00:00,492475.0,9984.0,8635.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1002,5.0,1.0,1.0,2008/2009,1.0,2008-08-16 00:00:00,492477.0,7947.0,9985.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1003,7.0,1.0,1.0,2008/2009,1.0,2008-08-16 00:00:00,492479.0,9999.0,8571.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1004,8.0,1.0,1.0,2008/2009,1.0,2008-08-16 00:00:00,492480.0,4049.0,9996.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,92.0,1.0,1.0,2008/2009,19.0,2009-02-18 00:00:00,492801.0,9991.0,8635.0,1.0,...,54.0,40.0,46.0,42.0,58.0,10.0,7.0,15.0,13.0,8.0
996,92.0,1.0,1.0,2008/2009,19.0,2009-02-18 00:00:00,492801.0,9991.0,8635.0,1.0,...,52.0,53.0,40.0,49.0,50.0,9.0,14.0,13.0,12.0,9.0
997,92.0,1.0,1.0,2008/2009,19.0,2009-02-18 00:00:00,492801.0,9991.0,8635.0,1.0,...,40.0,43.0,55.0,69.0,62.0,7.0,10.0,7.0,9.0,9.0
998,92.0,1.0,1.0,2008/2009,19.0,2009-02-18 00:00:00,492801.0,9991.0,8635.0,1.0,...,36.0,45.0,63.0,69.0,68.0,8.0,8.0,12.0,12.0,6.0


In [34]:
("a", "b") + ("d", "c")

('a', 'b', 'd', 'c')

In [53]:


# Connexion à la base de données SQLite
cursor = conn.cursor()

# Commande SQL pour créer la vue
create_view_query = f"""
CREATE VIEW player_stats_before_match AS
SELECT
    p.player_api_id,
    p.date,
    p.overall_rating,
    p.potential,
    m.match_api_id,
    m.date
FROM
    Player_Attributes p
    JOIN Match m ON p.player_api_id IN (m.home_player_1, m.home_player_2, m.home_player_3, m.home_player_4, m.home_player_5, m.home_player_6, m.home_player_7, m.home_player_8, m.home_player_9, m.home_player_10, m.home_player_11, m.away_player_1, m.away_player_2, m.away_player_3, m.away_player_4, m.away_player_5, m.away_player_6, m.away_player_7, m.away_player_8, m.away_player_9, m.away_player_10, m.away_player_11)
WHERE
    p.date = (
        SELECT MAX(p2.date)
        FROM Player_Attributes p2
        WHERE p2.player_api_id = p.player_api_id AND p2.date < m.date
    );
"""


# Commande SQL pour supprimer la vue si elle existe déjà
drop_view_query = "DROP VIEW IF EXISTS player_stats_before_match;"

# Exécution de la commande pour supprimer la vue
cursor.execute(drop_view_query)
# Exécution de la commande pour créer la vue
cursor.execute(create_view_query)

# Sauvegarder (commit) les modifications et fermer la connexion
conn.commit()


In [54]:
# Charger les données depuis la vue SQL
query = """
SELECT *
FROM player_stats_before_match
"""
df = pd.read_sql_query(query, conn)

# Charger la table des matchs
matches = pd.read_sql_query("SELECT * FROM match", conn)


In [ ]:
create_aggregate_view_query = f"""
CREATE VIEW match_team_ratings AS
SELECT
    m.match_api_id,
    m.home_team_api_id AS team_api_id,
    SUM(p.overall_rating) AS home_team_rating
FROM
    player_stats_before_match p
    JOIN Match m ON p.match_api_id = m.match_api_id AND p.player_api_id IN (m.home_player_1, m.home_player_2, m.home_player_3, m.home_player_4, m.home_player_5, m.home_player_6, m.home_player_7, m.home_player_8, m.home_player_9, m.home_player_10, m.home_player_11)
GROUP BY
    m.match_api_id,
    m.home_team_api_id
UNION ALL
SELECT
    m.match_api_id,
    m.away_team_api_id AS team_api_id,
    SUM(p.overall_rating) AS away_team_rating
FROM
    player_stats_before_match p
    JOIN Match m ON p.match_api_id = m.match_api_id AND p.player_api_id IN (m.away_player_1, m.away_player_2, m.away_player_3, m.away_player_4, m.away_player_5, m.away_player_6, m.away_player_7, m.away_player_8, m.away_player_9, m.away_player_10, m.away_player_11)
GROUP BY
    m.match_api_id,
    m.away_team_api_id;
"""

# Exécution de la commande pour créer la vue
cursor.execute(create_aggregate_view_query)

# Sauvegarder (commit) les modifications
conn.commit()

In [42]:
matches_df = pd.read_sql_query("SELECT * FROM match", conn)
player_attributes_df = pd.read_sql_query("SELECT * FROM player_Attributes", conn)

In [46]:
player_attributes_df["player_api_id"][0]

505942

In [50]:
matches_df[matches_df["home_player_3"] == player_attributes_df["player_api_id"][0]]

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
19317,19318,17642,17642,2014/2015,33,2015-05-18 00:00:00,1750765,8613,10214,0,...,NaN,3.0,3.3,2.5,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
import dask.dataframe as dd

# Convert pandas dataframes to dask dataframes
df_match_dask = dd.from_pandas(matchs, npartitions=10)
df_joueur_dask = dd.from_pandas(player_attributes, npartitions=10)

# Drop rows with null values in 'date' column and convert 'date' column to datetime
df_match_dask = df_match_dask.dropna(subset=['date'])
df_match_dask['date'] = dd.to_datetime(df_match_dask['date'])

df_joueur_dask = df_joueur_dask.dropna(subset=['date'])
df_joueur_dask['date'] = dd.to_datetime(df_joueur_dask['date'])

# Convert 'player_api_id' to float in df_joueur_dask to match the type in df_match_dask
df_joueur_dask['player_api_id'] = df_joueur_dask['player_api_id'].astype(float)

for joueur in home_player_cols:
    df_temp = dd.merge(df_match_dask, df_joueur_dask,  how='outer', left_on=[joueur], right_on = ['player_api_id'], suffixes=('', '_'+joueur))
    df_temp = df_temp.set_index('date')  # This will sort the DataFrame by 'date'
    df_temp = df_temp.fillna(method='ffill')
    df_match_dask = df_temp[df_temp.index.notnull()]

for joueur in away_player_cols:
    df_temp = dd.merge(df_match_dask, df_joueur_dask,  how='outer', left_on=[joueur], right_on = ['player_api_id'], suffixes=('', '_'+joueur))
    df_temp = df_temp.set_index('date')  # This will sort the DataFrame by 'date'
    df_temp = df_temp.fillna(method='ffill')
    df_match_dask = df_temp[df_temp.index.notnull()]

# Convert back to pandas dataframe if needed
df_match = df_match_dask.compute()

C:\Users\jdelavande\AppData\Local\Temp\ipykernel_23232\1564103119.py:20: FutureWarning: the 'method' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'method'
Use ffill or bfill instead.
  df_temp = df_temp.fillna(method='ffill')
c:\Users\jdelavande\Projects\optim-sportbet\venv\lib\site-packages\dask\dataframe\core.py:1896: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  meta = self._meta_nonempty.fillna(


NotImplementedError: dask.array<notnull, shape=(nan,), dtype=bool, chunksize=(nan,), chunktype=numpy.ndarray>